In [1]:
from google.colab import drive
import os
import pandas as pd
import numpy as np
import pickle
import re

drive.mount("/content/drive")
os.chdir("/content/drive/My Drive/BOAZ Conference/final_result")

Mounted at /content/drive


In [2]:
!pip install sentence_transformers
!pip install umap-learn
!pip install kss

     |████████████████████████████████| 92kB 5.1MB/s 
     |████████████████████████████████| 2.5MB 28.9MB/s 
     |████████████████████████████████| 1.2MB 21.8MB/s 
     |████████████████████████████████| 901kB 52.3MB/s 
     |████████████████████████████████| 3.3MB 25.4MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-cp37-none-any.whl size=126711 sha256=1938456a9120e3ecec6a88759b6a82eed22e352188672b6302f49146e79a8b25
  Stored in directory: /root/.cache/pip/wheels/38/d2/98/d191289a877a34c68aa67e05179521e060f96394a3e9336be6
Successfully built sentence-transformers
ERROR: transformers 4.8.2 has requirement huggingface-hub==0.0.12, but you'll have huggingface-hub 0.0.13 which is incompatible.
     |████████████████████████████████| 81kB 4.3MB/s 
     |████████████████████████████████| 1.2MB 12.2MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.1-cp37-none-any.whl size=76569 sha256=ecaacf119d5a569db92d3948ec60f7c6e13d8625b28a3203b2ba4a63649

In [3]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sentence_transformers import SentenceTransformer
import itertools

In [4]:
with open('umap_dim.pkl','rb') as f:
  umap = pickle.load(f)

total_df = pd.read_pickle('fined_cm_rocket_demoday.pkl')
umap_embeddings = np.load('fined_sbert_umap_vec.npy')
model_name = 'sent_bert'
emb_model = SentenceTransformer(model_name)

In [5]:
import nltk
from nltk.corpus import stopwords
import kss

nltk.download('stopwords')

eng_stopwords = stopwords.words('english')
eng_stopwords = [word for word in eng_stopwords if len(word) > 1]
eng_stopwords = [r'\b'+word.replace("'", "\b'")+r'\b' for word in eng_stopwords]

stop_words = ['2020년 12월', '전년동기', '전년대비', '연결기준', '대비', '매출액', '영업이익', '당기순이익', 
                '별도기준', '당기', '경영실적' ,'전년동기', '연결대상', '종속회사', '전년', '동사', '흑자전환', '적자전환',
               '성장', '국내', '매출', '기반', '주요', '증가', '영위'  '진행', '지속', '종속',
              '관련', '상장', '개발',  '악화', '축소', '사람', '세상', '가치', '생각', '판매', '부진',
              '사회', '변화', '목표', '경험', '정보', '혁신', '의미', '소통', '행복', '아이디어', '노력', 
              '시작', '문화', '시간', '시작', '준비', '한국', '인재', '기획', '선정', '전문가', '이상', '영업', '확대',
              '수익', '영향', '개선', '코로나','목적', '하락', '실적', '부담', '확대', '코스닥', '감소', 
              '손실', '세계', '최고', '자신', '필요', '고민', '문제', '지원', '관리', '동기', '유지', '글로벌', '세계', 
              '보유', '사랑', '신규', '사원수', '자본금', '규모', '위치', '서울', '중소', '강남구', '마포구', '강소', 
              '역삼동', '구로구', '벤처', '성동구', '서초구', '노동부', '빌딩', '가산동', '고용', '송파구', '금천구']
stop_words += eng_stopwords
remove_words = "("+"|".join(stop_words)+")"

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
class document_embedding():
  def __init__(self, text, remove_words):

    self.text = text
    self.remove_words = remove_words

  def preprocessing(self, x):
    assert len(x) > 10, '10글자 이상 입력하세요.' 
    text = re.sub('안녕하세요\S*', '', x)
    text = re.sub('\S+[.]*%\s*\S+', '', text)
    text = re.sub('\d+년\w*\s*|\d+월\w*\s*|\d+일\w*\s*', '', text)
    text = re.sub('[-=+,#/\?:^$@*\"※~&%ㆍ!』/\|\\‘|\(\)\[\]\<\>`\'…》]', ' ', text)
    text = re.sub('[\W|0-9]+\s{2,}','', text)
    text = re.sub(r'\\[[^]]*\\]' ,'', text)
    text = re.sub('【[^】]*】', '', text)
    text = re.sub('●\s+[A-Z|a-z]+','' , text)
    text = re.sub("(http|ftp|https)://(?:[-\w.]|(?:\da-fA-F]{2}))+\S+", '' , text)
    text = re.sub('\w+@\S+', '', text)
    text = re.sub('\xa0', ' ', text)
    text = re.sub('[^가-힣|a-zA-Z|\s|0-9|.]+', '', text)
    text = re.sub('\d[.]\s*', '', text)
    text = re.sub(r'(www[.]).+([.]com)', '', text)
    text = re.sub('(www[.])\S*', '', text)
    text_list = kss.split_sentences(text)
    text_list = list(map(lambda x: re.sub('(\w*\s*)'+remove_words+'(\w*\s*)', '', x), text_list))
    text_list = list(map(lambda x: re.sub('\s{2,}', ' ', x), text_list))
    text_list = list(map(lambda x: x.strip(), text_list))
    text_list = ' '.join(text_list)
    return text_list

  def get_emb_reduction(self, emb_model, reduction_model):
    fined_text = self.preprocessing(self.text)
    text_vec = emb_model.encode(fined_text).reshape(1,-1)
    umap_vec = reduction_model.transform(text_vec)
    return umap_vec

In [7]:
def identify_tokenizer(text):
  return text


def c_tf_idf(documents, m, ngram_range=(1, 1)):
    count = CountVectorizer(ngram_range=ngram_range, tokenizer=identify_tokenizer, lowercase=False).fit(documents)
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)

    return tf_idf, count

def extract_top_n_words_per_topic(tf_idf, count, df,n=20):
    words = count.get_feature_names()
    labels = list(df.label)
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    return top_n_words

def extract_topic_sizes(df):
    topic_sizes = (df.groupby(['label'])
                     .fined_token
                     .count()
                     .reset_index()
                     .rename({"label": "Topic", "fined_token": "Size"}, axis='columns')
                     .sort_values("Size", ascending=False))
    return topic_sizes

def get_cluster_words(df, X, cluster_num):
  agg_model = AgglomerativeClustering(n_clusters=cluster_num).fit(X)
  pred = agg_model.labels_
  if len(pred) > len(df):
    df['label'] = pred[:-1]
  else:
    df['label'] = pred

  cal_score = calinski_harabasz_score(X, pred)
  sil_score = silhouette_score(X, pred, metric='cosine')
  print('cal_score :{0:0.4f} sil_score : {1:0.4f}'.format(cal_score, sil_score))

  doc_per_cls = df.groupby(['label'], as_index=False).agg({'fined_text': 'sum','fined_token': 'sum'})
  tf_idf, count = c_tf_idf(doc_per_cls.fined_token.values, m=len(df))
  top_n_words = extract_top_n_words_per_topic(tf_idf, count, doc_per_cls, n=20)
  topic_sizes = extract_topic_sizes(df)

  
  key = pred[-1]
  value = [word[0] for word in top_n_words[key]]
  print(key, ':', value)

def get_neighbors(df, doc_vec):
  agg_model = AgglomerativeClustering(n_clusters=None, distance_threshold=0.5).fit(doc_vec)
  key = agg_model.labels_[-1]
  ind = agg_model.labels_ == key
  result = total_df.loc[ind[:-1], ['company_name', 'train_data']]
  return result

In [8]:
# input에 텍스트 넣어주면
# 1) 38개의 산업 분류 중 입력된 설명에 해당하는 Label 반환 & Label 키워드 반환
# 2) 입력된 설명과 유사한 기업들로 구성된 DataFrame 반환 (neighbor_df)

test_text = input()

CJ그룹의 디저트 카페 프랜차이즈 기업이다. 2002년 신촌점을 시작으로 2018년 현재 전국에 1,000여개의 점포를 운영하고 있다


In [9]:
new_doc = document_embedding(test_text, remove_words)
new_doc_vec = new_doc.get_emb_reduction(emb_model, umap)
total_vec = np.concatenate([umap_embeddings, new_doc_vec], axis=0)
get_cluster_words(total_df, total_vec, 38)
neighbor_df = get_neighbors(total_df, total_vec).rename(columns={'company_name':'회사명', 'train_data':'기업 설명'}).reset_index(drop=True)
neighbor_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Possible nested set at position 3
  


cal_score :6785.9234 sil_score : 0.6307


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


0 : ['식품', '커피', '푸드', '음식', '유통', '소비자', '건강', '농업', '가공', '배달', '사료', '외식', '브랜드', '온라인', '생산', '카페', '매장', '주문', '음료', '맛집']


,회사명,기업 설명
0,벤디스,벤디스 대표 조정호 www.VENDYS.co.kr 는 설립된 기업 O2O Onlin...
1,리치빔,주 리치빔은 설립하여 견고하고 실질적인 연구와 지식을 바탕으로 끊임없는 새로운 지식...
2,란체스터에프앤비,란체스터에프앤비는 싸움의고수 처음으로 1인 보쌈의 시대를 열어 현재 190개의 가맹...
3,얌테이블,수산신선식품 온라인커머스 1등 스타트업얌테이블은 거제 산지에 FPC Fish Pro...
4,본아이에프,본그룹 본아이에프 본푸드서비스 순수본 본에프디 대학로의 작은 본죽 1호점 만에 한식...
